In [1]:
# 예외가 발생하더라도 정리 코드를 실행해야하는 경우

def try_finally_example(filename):
    print('* 파일 열기')
    handle = open(filename, encoding='utf-8') # OSError가 발생할 수 있음
    try:
        print('* 데이터 읽기')
        return handle.read() # UnicodeDecodeError가 발생 할 수 있음
    
    finally:
        print('* close() 호출')
        handle.close() # try 블록이 실행된 다음에는 항상 이 블록이 실행됨

In [2]:
file_name = 'random_data.txt'

with open(file_name, 'wb') as f:
    f.write(b'\xf1\xf2\xf3\xf4\xf5') # 잘못된 utf-8 이진 문자열
    
data = try_finally_example(file_name)


* 파일 열기
* 데이터 읽기
* close() 호출


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xf1 in position 0: invalid continuation byte

In [3]:
try_finally_example('does_not_exist.txt') # try이전에 호출해서 finally가 동작하지 않도록 구현

* 파일 열기


FileNotFoundError: [Errno 2] No such file or directory: 'does_not_exist.txt'

문자열에서 Json 딕셔너리 데이터를 읽어온 후 어떤 키에 해당하는 값을 반환하고 싶다

In [6]:
import json

def load_json_key(data, key):
    try:
        print('* Json 데이터 읽기')
        result_dict = json.loads(data) # ValueError가 발생할 수 있음 
        
    except ValueError as e:
        print("* ValueError 처리")
        raise KeyError(key) from e
    
    else:
        print("* 키 검색")
        return result_dict[key] # KeyError가 발생 할 수 있음 


In [8]:
assert load_json_key('{"foo": "bar"}', 'foo') == 'bar' # except가 안되기 때문에 else구문 실행

* Json 데이터 읽기
* 키 검색


In [11]:
load_json_key('{"foo": bad payload}', 'foo') # 올바른 Json이 아님 

# json.loads가 디코딩하는 중간에 ValueError를 발생시킨다
# except 블럭에 의해 처리된다.

* Json 데이터 읽기
* ValueError 처리


KeyError: 'foo'

In [12]:
load_json_key('{"foo": "bar"}', '존재하지 않음')

# return result_dict[key]에서 예외가 발생해서 
# '존재하지 않음'이라는 key는 없다고 볼 수 있다

* Json 데이터 읽기
* 키 검색


KeyError: '존재하지 않음'

try except else finally문을 모두 사용하면 복잡적인 문장 모두 처리 가능하다

In [19]:
# 정상 실행되는 경우 try - else - finally 블록이 실행된다. 

UNDEFINED = object()

def divide_json(path):
    print('* 파일 열기')
    handle = open(path, 'r+') # OSError가 발생 할 수 있음
    
    try:
        print("* 데이터 읽기")
        data = handle.read()
        print("* JSON 데이터 읽기")
        op = json.loads(data)
        print('* 계산 수행')
        value = (
            op['numerator'] /
            op['denominator']) # op['denominator']가 0이면 ZeroDivisionError
        print(value)
        
    except ZeroDivisionError as e:
        print('* ZeroDivisionError 처리')
        return UNDEFINED
    else:
        print('* 계산 결과 쓰기')
        op['result'] = value
        result = json.dumps(op)
        handle.seek(0) # OSError 발생 가능
        handle.write(result) # OSError 발생 가능
        return value
    finally:
        print("* close() 호출")
        handle.close()

In [20]:
temp_path = 'random_data.json'

with open(temp_path, 'w') as f:
    f.write('{"numerator" : 1, "denominator": 10}')
    
assert divide_json(temp_path) == 0.1

* 파일 열기
* 데이터 읽기
* JSON 데이터 읽기
* 계산 수행
0.1
* 계산 결과 쓰기
* close() 호출


In [21]:
with open(temp_path, 'w') as f:
    f.write('{"numerator": 1, "denominator": 0}') # ZeroDivisionError
    
assert divide_json(temp_path) is UNDEFINED

* 파일 열기
* 데이터 읽기
* JSON 데이터 읽기
* 계산 수행
* ZeroDivisionError 처리
* close() 호출


In [22]:
with open(temp_path, 'w') as f:
    f.write('{"numerator": 1 bad data') # json 형식이 아님
    
divide_json(temp_path) # finally 까지 호출

* 파일 열기
* 데이터 읽기
* JSON 데이터 읽기
* close() 호출


JSONDecodeError: Expecting ',' delimiter: line 1 column 17 (char 16)

In [ ]:
from threading import Lock

lock = Lock()

with lock:
    # 어떤 불변 조건을 유지하면서 작업 수행
    ...
    


Lock 클래서가 with문을 적절히 활성화 해주므로 위 예제는 다음 try/finally 구조와 동등하다


In [ ]:
lock.acquire()

try:
    # 어떤 불변 조건을 유지하면서 작업을 수행한다
    ...
finally:
    lock.release()

이 경우에는 with 문쪽이 더 낫다 try/finally 구조의 반복사용을 피하고 acquire에 대응하는 release 실수로 빠뜨리는 경우를 방지 할 수 있다

contextlib 내장 모듈을 사용하면 객체나 함수를 with 문에서 쉽게 쓸 수 있다

예를 들어 어떤 코드 영역에서 디버깅 관련 로그를 더 많이 남기고 싶다면
다음 코드는 두 단계의 심각성 수준에서 디버깅 로그를 남기는 함수를 정의한다

In [23]:
import logging
def my_function():
    logging.debug('디버깅 데이터')
    logging.error('이 부분은 오류 로그')
    logging.debug('추가 디버깅 데이터 ')
    
my_function() # 프로그램의 디폴드 로그 수준은 Warning이라 이 함수를 실행하면 오류 메시지만 화면에 출력된다

ERROR:root:이 부분은 오류 로그


 컨텍스트 매니저를 정의하면 이 함수의 로그수준을 일시적으로 높일 수 있다

 with 블록을 실행하기전에 로그 심각성 수준을 높이고 블록 실행 직후에 이전 수준으로 회복

In [25]:
from contextlib import contextmanager

@contextmanager
def debug_logging(level):
    logger = logging.getLogger()
    
    old_level = logger.getEffectiveLevel()
    logger.setLevel(level)
    try:
        yield # with 블록의 내용이 실행되는 부분을 저장한다 
    finally:
        logger.setLevel(old_level)
        

In [32]:
with debug_logging(logging.DEBUG):
    print('* 내부: ')
    my_function()
    
print('* 외부:')
my_function()

DEBUG:root:디버깅 데이터
ERROR:root:이 부분은 오류 로그
DEBUG:root:추가 디버깅 데이터 
ERROR:root:이 부분은 오류 로그


* 내부: 
* 외부:
